In [1]:
# Importing libraries
import pandas as pd

C:\Users\OPEF\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# preparing al jazeera corpus

In [9]:
# Importing corpus
df = pd.read_csv('test_snippet_2.csv')

# Print head
#df.head()

In [3]:
#removing empty lines
df.dropna()

,title,date,url,sentence
0,South Korea wants North punished over football...,2019-10-18,https://www.aljazeera.com/sports/2019/10/18/so...,South Korea’s football association accused the...
1,South Korea wants North punished over football...,2019-10-18,https://www.aljazeera.com/sports/2019/10/18/so...,The South Korean football association (KFA) ha...
2,South Korea wants North punished over football...,2019-10-18,https://www.aljazeera.com/sports/2019/10/18/so...,The Seoul-based KFA’s letter to the Asian Foot...
3,South Korea wants North punished over football...,2019-10-18,https://www.aljazeera.com/sports/2019/10/18/so...,"The game, which was cast into media darkness b..."
4,South Korea wants North punished over football...,2019-10-18,https://www.aljazeera.com/sports/2019/10/18/so...,"Upon their return, South Korean players said t..."
...,...,...,...,...
87481,Erdogan open to meeting al-Assad but not to wi...,2023-07-17,https://www.aljazeera.com/news/2023/7/17/erdog...,While tensions between the two countries remai...
87482,Erdogan open to meeting al-Assad but not to wi...,2023-07-17,https://www.aljazeera.com/news/2023/7/17/erdog...,The restoration of ties with Damascus quickene...
87483,Erdogan open to meeting al-Assad but not to wi...,2023-07-17,https://www.aljazeera.com/news/2023/7/17/erdog...,"On Sunday, Iraq’s prime minister held talks wi..."
87484,Erdogan open to meeting al-Assad but not to wi...,2023-07-17,https://www.aljazeera.com/news/2023/7/17/erdog...,The decision to allow Syria back into the Arab...


In [4]:
#removing contentless Al Jazeera Banner 
df.drop(df[df['sentence'].str.contains('Follow Al Jazeera English:')| df['sentence'].str.contains('Al Jazeera English:')].index, inplace=True)

In [5]:
#removing incorrect formated lines
try:
    # Überprüfe und ersetze ungültige Datumsangaben
    df['date'] = pd.to_datetime(df['date'], errors='coerce')

    # Selektiere Zeilen mit ungültigen Datumsangaben
    invalid_rows = df[df['date'].isna()]

    # Überprüfe, ob ungültige Zeilen gefunden wurden
    if not invalid_rows.empty:
        print(f"{len(invalid_rows)} fehlerhafte Zeilen gefunden und entfernt.")
        # Entferne Zeilen mit NaT in der Spalte 'date'
        df = df.dropna(subset=['date'])
    else:
        print("Alle Datumsangaben korrekt.")
except Exception as e:
    print(f"Fehler beim Überprüfen der Datumsangaben: {e}")

293 fehlerhafte Zeilen gefunden und entfernt.


In [6]:
#Adding coloumn year
df['year'] = pd.to_datetime(df['date']).dt.year

In [7]:
# remove liveblog entries without usefull content
df_check = df.groupby(['title','year','url'])['sentence'].agg(lambda x: ' '.join(str(val) for val in x if pd.notna(val))).reset_index()
df_check.drop(df_check[df_check['url'].str.contains('https://www.aljazeera.com/sports/liveblog')|df_check['url'].str.contains('https://www.aljazeera.com/news/liveblog')].index, inplace=True)

In [8]:
# adding relevance scores to secure that articels are related to our topic 
# keywords were determined by building a semantic field for the term 'fifa'
# context based differentiation between qatar and teh other closer related terms from the semantic field

def calculate_score_fifa(text):
    keywords = ['fifa', 'world cup', 'blatter', 'infantino']
    # Konvertiere Text und Keywords in Kleinbuchstaben für case-sensitive Suche
    text_lower = text.lower()
    score = sum(text_lower.count(keyword) for keyword in keywords)
    return score

def calculate_score_qatar(text):
    keywords = ['qatar']
    # Konvertiere Text und Keywords in Kleinbuchstaben für case-sensitive Suche
    text_lower = text.lower()
    score = sum(text_lower.count(keyword) for keyword in keywords)
    return score

# Konvertiere alle Zeichen in der Spalte 'Text' in Kleinbuchstaben
df_check['sentence'] = df_check['sentence'].str.lower()

# Scores für Tehemnzugehörigkeit und Länge berechnen
df_check['score_relevance_fifa'] = df_check['sentence'].apply(calculate_score_fifa)
df_check['score_relevance_qatar'] = df_check['sentence'].apply(calculate_score_qatar)
df_check['score_article_length'] = df_check['sentence'].str.len()

#Ergebnis-Output zu Evaluation der notwendigen thresholds
output_check = df_check[['url', 'score_relevance_fifa', 'score_relevance_qatar','score_article_length']].reset_index(drop=True)
output_check.to_csv('output_check.csv', index=False)

filtered_df = df_check[(df_check['score_article_length'] < 2000) & (df_check['score_relevance_fifa'] > 0) & (df_check['score_relevance_qatar'] > 0) |
                 (df_check['score_article_length'] >= 2000) & (df_check['score_article_length'] < 25000) & (df_check['score_relevance_fifa'] > 1) & (df_check['score_relevance_qatar'] > 1)]

filtered_df.reset_index(drop=True, inplace=True)

grouped_filtered_df = filtered_df[['title', 'year', 'sentence']]
#grouped_filtered_df.to_csv('grouped_filtered_df.csv', index=False)

# preparing guardian corpus

# combining both corpora